In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
'''
df = pd.read_csv('./Refined_SPADE_Year20182019_Ver20210307.csv')
df = df.set_index('StudyID')
print(df.shape)
print(df.columns)
feature_columns_cont = [
     'ed_inp_delta'
    ,'ed_duration'
    ,'elix_score'
    ,'age_val'
]
feature_columns_cat = [
     'img_flags'
    ,'con_flags'
]
print(df.PatientSSN.unique().shape)
feature_columns = [
     'ed_duration'
    ,'ed_inp_delta'
    ,'age_val'
    ,'img_flags'
    ,'con_flags'
    ,'elix_score'
]
df[feature_columns]
'''
df = pd.read_csv("input/input.csv")
df = df.set_index('study_ids')
df = df.drop('smk_flags_man', axis=1)
df = df.drop('smk_flags_db', axis=1)
df = df[df.mod_label != 'INVALID']

feature_columns_cont = [
     'ed_inp_delta'
    ,'ed_duration'
    ,'elix_score'
    ,'age_val'
]
feature_columns_cat = [
     'img_flags'
    ,'con_flags'
]
feature_columns = [
     'ed_duration'
    ,'ed_inp_delta'
    ,'age_val'
    ,'img_flags'
    ,'con_flags'
    ,'elix_score'
]
label_column = 'mod_label'

df['ed_inp_delta'] = MinMaxScaler().fit_transform(df['ed_inp_delta'].values.reshape(-1,1))
df['ed_duration'] = MinMaxScaler().fit_transform(df['ed_duration'].values.reshape(-1,1))
df['elix_score'] = MinMaxScaler().fit_transform(df['elix_score'].values.reshape(-1,1))
df['age_val'] = MinMaxScaler().fit_transform(df['age_val'].values.reshape(-1,1))

In [ ]:
dxdf = pd.read_csv('./EDDiag.csv')
dxdf = dxdf.set_index('StudyID')
#Get the icd10codes for each studyID as a flat row
#Just take first 10 here to get an idea for how it looks
print(dxdf.head())
print()
for i in df.index.values[:10]:
    if i in dxdf.index.values:
        print(i,dxdf.loc[i].to_numpy().reshape(-1,2)[:,0])
print()     
print('The number of unique ICDs we have:', end=' ')
print(dxdf.ICD10Code.unique().shape[0])
print("The number of StudyIDs we have ICDs for:", end=' ')
print(dxdf.index.values.shape[0])
print("The number of studyIDs in the SPADE set for which we don't have an ICD:", end=" ")
print(len([i for i in df.index.values if i not in dxdf.index.values]))
print("The number of studyIDs we have in the SPADE set:", end=" ")
print(df.index.values.shape[0])

In [15]:
studyicds = dict()
for i in df.index.values:
    if i in dxdf.index.values:
        studyicds[i] = dxdf.loc[i].to_numpy().reshape(-1,2)[:,0]
    else:
        studyicds[i] = None

In [16]:
#Binarize ICDs
from sklearn.preprocessing import MultiLabelBinarizer

In [17]:
mlb = MultiLabelBinarizer()
mlb.fit(np.asarray([x for x in studyicds.values() if type(x) == np.ndarray], dtype='object'))
sizetopad = mlb.transform(np.asarray([x for x in studyicds.values() if type(x) == np.ndarray], dtype='object'))[0].shape[0]

In [18]:
binarizedicds = dict()
for key,values in studyicds.items():
    if type(values) == np.ndarray:
        binarizedicds[key] = mlb.transform(values.reshape(1,-1)).reshape(-1, )
    else:
        binarizedicds[key] = np.zeros(sizetopad, dtype=int)


In [19]:
dxdf = pd.DataFrame.from_dict(binarizedicds, orient='index')
dfx = df.join(dxdf, how='inner')

In [20]:
from sklearn.model_selection import train_test_split
TEST_SIZE = 0.15
RANDOM_STATE_SEED = None

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

test_aucs = []
train_aucs = []
test_reports = []
train_reports = []
for i in range(100):
    eX_train, eX_test, ey_train, ey_test = train_test_split(dfx.drop(label_column, axis=1), \
                                                            dfx[label_column], \
                                                            test_size=TEST_SIZE, random_state=RANDOM_STATE_SEED)
    model = LogisticRegression(solver='liblinear', penalty='l2', C=0.7, max_iter=1000, fit_intercept = False)
    etest = eX_test.iloc[:,6:]
    etrain = eX_train.iloc[:,6:]
    model.fit(etrain, ey_train)
    test_probas  = model.predict_proba(etest)
    test_preds = model.predict(etest)
    train_preds = model.predict(etrain)
    train_probas = model.predict_proba(etrain)
    testauc = roc_auc_score(ey_test, test_probas[:,1])
    trainauc = roc_auc_score(ey_train, train_probas[:,1])
    test_aucs += [testauc]
    train_aucs += [trainauc]
    test_reports += [classification_report(ey_test, test_preds, output_dict=True)]
    train_reports += [classification_report(ey_train, train_preds, output_dict=True)]

print(f" Average Test AUC \u00B1 \u03C3 (100 iterations): " +\
      f"{np.average(test_aucs):0.3f} \u00B1 {np.std(test_aucs):0.3f}")
print(f"Average Train AUC \u00B1 \u03C3 (100 iterations): " +\
      f"{np.average(train_aucs):0.3f} \u00B1 {np.std(train_aucs):0.3f}")

 Average Test AUC ± σ (100 iterations): 0.513 ± 0.082
Average Train AUC ± σ (100 iterations): 0.808 ± 0.013


In [40]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)

pca.fit(dfx.drop(label_column,axis=1).iloc[:,6:])

test_aucs = []
train_aucs = []
test_reports = []
train_reports = []
for i in range(100):
    eX_train, eX_test, ey_train, ey_test = train_test_split(dfx.drop(label_column, axis=1), \
                                                            dfx[label_column], \
                                                            test_size=TEST_SIZE, random_state=RANDOM_STATE_SEED)
    model = LogisticRegression(solver='liblinear', penalty='l2', C=0.7, max_iter=1000, fit_intercept = False)
    etest = np.hstack([eX_test.iloc[:,:6], pca.transform(eX_test.iloc[:,6:])])
    etrain = np.hstack([eX_train.iloc[:,:6], pca.transform(eX_train.iloc[:,6:])])
    model.fit(etrain, ey_train)
    test_probas  = model.predict_proba(etest)
    test_preds = model.predict(etest)
    train_preds = model.predict(etrain)
    train_probas = model.predict_proba(etrain)
    testauc = roc_auc_score(ey_test, test_probas[:,1])
    trainauc = roc_auc_score(ey_train, train_probas[:,1])
    test_aucs += [testauc]
    train_aucs += [trainauc]
    test_reports += [classification_report(ey_test, test_preds, output_dict=True)]
    train_reports += [classification_report(ey_train, train_preds, output_dict=True)]

print(f" Average Test AUC \u00B1 \u03C3 (100 iterations): " +\
      f"{np.average(test_aucs):0.3f} \u00B1 {np.std(test_aucs):0.3f}")
print(f"Average Train AUC \u00B1 \u03C3 (100 iterations): " +\
      f"{np.average(train_aucs):0.3f} \u00B1 {np.std(train_aucs):0.3f}")

 Average Test AUC ± σ (100 iterations): 0.684 ± 0.091
Average Train AUC ± σ (100 iterations): 0.716 ± 0.015
